In [1]:
from marubatsu import List1dBoard, Marubatsu

mb = Marubatsu(boardclass=List1dBoard)
mb.cmove(0, 0)
mb.cmove(1, 0)
mb.cmove(0, 1)
mb.cmove(2, 0)
mb.cmove(1, 2)
mb.cmove(1, 1)
mb.cmove(0, 2)
print(mb)

winner o
oxx
ox.
Oo.



In [2]:
from util import benchmark

boardclass = List1dBoard
for count_linemark in [False, True]:
    print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
    benchmark(mbparams={"boardclass": boardclass, "count_linemark": count_linemark})
    print()

boardclass: List1dBoard, count_linemark False
ai2 VS ai2


100%|██████████| 50000/50000 [00:02<00:00, 18942.59it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:30<00:00, 1664.46it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 18.2 ms ±   1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

boardclass: List1dBoard, count_linemark True
ai2 VS ai2


100%|██████████| 50000/50000 [00:02<00:00, 17171.76it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:17<00:00, 2814.81it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 17.8 ms ±   0.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



In [3]:
print(mb)
x = 0
y = 0
print(mb.board.board[y::mb.BOARD_SIZE])
print(mb.board.board[x:(x + 1) * mb.BOARD_SIZE])
print(mb.board.board[::mb.BOARD_SIZE + 1])
print(mb.board.board[mb.BOARD_SIZE - 1:-1:mb.BOARD_SIZE - 1])

winner o
oxx
ox.
Oo.

['o', 'x', 'x']
['o', 'o', 'o']
['o', 'x', '.']
['o', 'x', 'x']


In [4]:
dx = 0
dy = 1
print([mb.board.board[y + i * dy + (x + i * dx) * mb.BOARD_SIZE] for i in range(mb.BOARD_SIZE)])
%timeit [mb.board.board[y + i * dy + (x + i * dx) * mb.BOARD_SIZE] for i in range(mb.BOARD_SIZE)]
%timeit mb.board.board[x:(x + 1) * mb.BOARD_SIZE]

['o', 'o', 'o']
391 ns ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
121 ns ± 1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
def is_winner(self, player, last_move):
    x, y = last_move // self.BOARD_SIZE, last_move % self.BOARD_SIZE
    if self.count_linemark:
        if self.rowcount[player][y] == self.BOARD_SIZE or \
        self.colcount[player][x] == self.BOARD_SIZE:
            return True
        # 左上から右下方向の判定
        if x == y and self.diacount[player][0] == self.BOARD_SIZE:
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            self.diacount[player][1] == self.BOARD_SIZE:
            return True
    else:
        winstr = player * self.BOARD_SIZE
        # 行と列の判定
        if "".join(self.board[y::self.BOARD_SIZE]) == winstr or \
           "".join(self.board[x * self.BOARD_SIZE:(x + 1) * self.BOARD_SIZE]) == winstr:
                return True
        # 左上から右下方向の判定
        if x == y and "".join(self.board[::self.BOARD_SIZE + 1]) == winstr:
            return True
        # 左下から右上方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            "".join(self.board[self.BOARD_SIZE - 1:-1:self.BOARD_SIZE - 1]) == winstr:
            return True
    
    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False  

List1dBoard.is_winner = is_winner

In [5]:
from copy import deepcopy

boardorig = deepcopy(mb.board.board)
print(mb)
rotboard = []
for i in range(mb.BOARD_SIZE):
    rotboard += mb.board.board[mb.BOARD_SIZE - 1 - i::mb.BOARD_SIZE]
mb.board.board = rotboard
print(mb)
mb.board.board = boardorig

winner o
oxx
ox.
Oo.

winner o
ooo
oxx
..x



In [7]:
print(mb)
for i in range(mb.BOARD_SIZE):
    print(mb.board.board[(i + 1) * mb.BOARD_SIZE - 1:i * mb.BOARD_SIZE - 1:-1])

winner o
oxx
ox.
Oo.

[]
['o', 'x', 'x']
['.', '.', 'x']


In [8]:
print(mb)
for i in range(mb.BOARD_SIZE):
    start = (i + 1) * mb.BOARD_SIZE - 1
    end = start - mb.BOARD_SIZE if i > 0 else None
    print(mb.board.board[start:end:-1])

winner o
oxx
ox.
Oo.

['o', 'o', 'o']
['o', 'x', 'x']
['.', '.', 'x']


In [9]:
boardorig = deepcopy(mb.board.board)
print(mb)
flipboard = []
for i in range(mb.BOARD_SIZE):
    start = (i + 1) * mb.BOARD_SIZE - 1
    end = start - mb.BOARD_SIZE if i > 0 else None    
    flipboard += mb.board.board[start:end:-1]
mb.board.board = flipboard
print(mb)
mb.board.board = boardorig

winner o
oxx
ox.
Oo.

winner o
oo.
ox.
Oxx



In [10]:
a = [1, 2, 3]
print(reversed(a))
print(list(reversed(a)))

[3, 2, 1]


In [11]:
print(a)
print(a.reverse())
print(a)

[1, 2, 3]
None
[3, 2, 1]


In [ ]:
def calc_same_hashables2(self, move=None):
    if move is None:
        hashables = set([self.board_to_hashable()])
    else:
        hashables = { self.board_to_hashable(): move }
    boardorig = self.board
    if move is not None:
        x, y = self.move_to_xy(move)
    for i in range(7):
        if i != 3:
            rotboard = []
            for i in range(self.BOARD_SIZE):
                rotboard += self.board[self.BOARD_SIZE - 1 - i::self.BOARD_SIZE]
            self.board = rotboard
        else:
            flipboard = []
            for i in range(self.BOARD_SIZE):
                start = (i + 1) * self.BOARD_SIZE - 1
                end = start - self.BOARD_SIZE if i > 0 else None  
                flipboard += self.board[start:end:-1]
            self.board = flipboard
        hashable = self.board_to_hashable()
        if move is None:
            hashables.add(hashable)
        else:
            if i == 3:
                y = self.BOARD_SIZE - y - 1
            else:
                x, y = self.BOARD_SIZE - y - 1, x
            hashables[hashable] = self.xy_to_move(x, y)
    self.board = boardorig
    return hashables

List1dBoard.calc_same_hashables2 = calc_same_hashables2

In [13]:
from pprint import pprint

move = mb.board.xy_to_move(0, 0)
h1 = mb.board.calc_same_hashables(move)
h2 = mb.board.calc_same_hashables2(move)
pprint(h1)
pprint(h2)
print(h1 == h2)


{'..xoxxooo': 8,
 '.oo.xoxxo': 8,
 'oo.ox.oxx': 6,
 'ooooxx..x': 2,
 'oooxxox..': 0,
 'oxxox.oo.': 0,
 'x..xxoooo': 6,
 'xxo.xo.oo': 2}
{'..xoxxooo': 8,
 '.oo.xoxxo': 8,
 'oo.ox.oxx': 6,
 'ooooxx..x': 2,
 'oooxxox..': 0,
 'oxxox.oo.': 0,
 'x..xxoooo': 6,
 'xxo.xo.oo': 2}
True


In [14]:
%timeit mb.board.calc_same_hashables(move)
%timeit mb.board.calc_same_hashables2(move)

21.1 μs ± 505 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
8.07 μs ± 32.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
def calc_same_hashables(self, move=None):
    if move is None:
        hashables = set([self.board_to_hashable()])
    else:
        hashables = { self.board_to_hashable(): move }
    boardorig = self.board
    if move is not None:
        x, y = self.move_to_xy(move)
    for i in range(7):
        if i != 3:
            rotboard = []
            for i in range(self.BOARD_SIZE):
                rotboard += self.board[self.BOARD_SIZE - 1 - i::self.BOARD_SIZE]
            self.board = rotboard
        else:
            flipboard = []
            for i in range(self.BOARD_SIZE):
                start = (i + 1) * self.BOARD_SIZE - 1
                end = start - self.BOARD_SIZE if i > 0 else None
                flipboard += self.board[start:end:-1]
            self.board = flipboard
        hashable = self.board_to_hashable()
        if move is None:
            hashables.add(hashable)
        else:
            if i == 3:
                y = self.BOARD_SIZE - y - 1
            else:
                x, y = self.BOARD_SIZE - y - 1, x
            hashables[hashable] = self.xy_to_move(x, y)
    self.board = boardorig
    return hashables

List1dBoard.calc_same_hashables = calc_same_hashables

In [16]:
boardclass = List1dBoard
for count_linemark in [False, True]:
    print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
    benchmark(mbparams={"boardclass": boardclass, "count_linemark": count_linemark})
    print()

boardclass: List1dBoard, count_linemark False
ai2 VS ai2


100%|██████████| 50000/50000 [00:02<00:00, 24249.15it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:27<00:00, 1843.54it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
  8.7 ms ±   0.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

boardclass: List1dBoard, count_linemark True
ai2 VS ai2


100%|██████████| 50000/50000 [00:02<00:00, 18801.86it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:17<00:00, 2890.13it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 10.1 ms ±   0.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

